In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output, State

import os
import base64
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bson.json_util import dumps
from pymongo import MongoClient


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "accuser" # Adjusted username to created username from previous module
password = "cameronlee" # Adjusted password to created password from previous module
shelter = AnimalShelter(username, password) # Call shelter


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({})) 





#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# Adding picture to dashboard
image_filename = 'GraziosoSalvareLogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    # Centering logo
    html.Center(html.Img(id='main-logo', src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='main-logo', height=150)),
    # Center unique identifier for dashboard
    html.Center(html.B(html.H1('Cameron Lee - SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        dcc.RadioItems( # radio buttons to filter by type
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'}, # Water filter
                {'label': 'Mountain Rescue', 'value': 'mountain'}, # Mountain filter
                {'label': 'Disaster Rescue', 'value': 'disaster'}, # Disaster filter
                {'label': 'Reset', 'value': 'reset'} # Shows all animals (unfiltered)
            ],
            value='reset' # Default filter
        )
    ),
    html.Hr(),
    dash_table.DataTable( # Data table
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        
        
        # Setting up dictionary features
        editable=False,
        row_selectable=True,
        column_selectable=True,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10,
        sort_action="native",
        sort_mode="multi",

    ),
    html.Br(),
     html.Hr(),
     html.Div(className='row',
              style={'display': 'flex'},
              children=[
                  html.Div( # Map display
                      id='map-id',
                      className='col s12 m6',
                  ),
                  html.Div( # Pie chart display
                      id='graph-id',
                      className='col s12 m6',
                  )
              ])
            
])

#############################################
# Interaction Between Components / Controller
#############################################


# ******************************* FILTERS *******************************

# Callback
@app.callback([Output('datatable-id', 'data'),
               Output('datatable-id', 'columns')],
              [Input('filter-type', 'value')]
)

# Function
def update_dashboard(filter_type):
    if (filter_type == 'mountain'): # Mountain rescue filter
        df = pd.DataFrame.from_records(shelter.read(
        {
            "animal_type": "Dog",
            "breed": {
                "$in": [
                    "German Shepherd",
                    "Alaskan Malamute",
                    "Old English Sheepdog",
                    "Siberian Husky",
                    "Rottweiler"
                ]
            },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {
                "$gte": 26.0,
                "$lte": 156.0
            }
        }))
        
    elif (filter_type == 'disaster'): # Disaster relief filter
        df = pd.DataFrame.from_records(shelter.read(
        {
            "breed": {
                "$in": [
                    "Doberman Pinscher",
                    "German Shepherd",
                    "Golden Retriever",
                    "Bloodhound",
                    "Rottweiler"
                ]
            },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {
                "$gte": 26.0
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks": {
                        "$lte": 156.0
                    }
                }
            ]
        }
        ))
    
    elif (filter_type == 'water'): # Water rescue filter
        df = pd.DataFrame.from_records(shelter.read(
        {
            "breed": {
                "$in": [
                    "Labrador Retriever Mix",
                    "Chesapeake Bay Retriever",
                    "Newfoundland"
                ]
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks": {
                        "$gte": 26.0
                    },
                    "$and": [
                        {
                            "age_upon_outcome_in_weeks": {
                                "$lte": 156.0
                            }
                        }
                    ]
                }
            ]
        }
        ))
        
    else: # Reset filter (DEFAULT)
        df = pd.DataFrame.from_records(shelter.read({}))
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
    
    return (data, columns)






# ******************************* HIGHLIGHT *******************************

# Callback
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)

# Function
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF' # Highlight color
    } for i in selected_columns]

# ******************************* PIE CHART *******************************

# Callback
@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'derived_viewport_data')]
)

# Function
def update_graph(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    
    return [
        dcc.Graph(figure = px.pie(dff, names='breed')
                 )
    ]

# ******************************* MAP *******************************

# Callback
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)

# Function
def update_map(viewData):

    dff = pd.DataFrame.from_dict(viewData)
    
    # Austin TX is at [30.75, -97.48]
    
    ### Geochart Variables ###
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75, -97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9]),
                    
                ])
            ])
        ])
    ]
    

app